# Data Owner

In [ ]:
!pip install syft==0.5.0
!pip install tenseal

     |████████████████████████████████| 504 kB 4.2 MB/s 
     |████████████████████████████████| 66 kB 1.7 MB/s 
     |████████████████████████████████| 789 kB 3.1 MB/s 
     |████████████████████████████████| 17.4 MB 9.6 MB/s 
     |████████████████████████████████| 4.9 MB 46.2 MB/s 
     |████████████████████████████████| 2.0 MB 43.2 MB/s 
     |████████████████████████████████| 43 kB 2.0 MB/s 
     |████████████████████████████████| 596 kB 36.8 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
     |████████████████████████████████| 856 kB 16.2 MB/s 
     |████████████████████████████████| 3.6 MB 18.5 MB/s 
     |████████████████████████████████| 54 kB 2.9 MB/s 
     |████████████████████████████████| 57 kB 5.2 MB/s 
     |████████████████████████████████| 804.1 MB 2.5 kB/s 
     |████████████████████████████████| 71 kB 4.2 MB/s 
     |████████████████████████████████| 36.1 MB 1.1 MB/s 
     |████████████████████████████████| 241 kB 68.0 MB/s 
  Created wheel for forbidd

In [ ]:
import os
import syft as sy
import tenseal as ts
import torch
import pandas as pd
import random
import numpy as np
import requests

import pytest
from time import time
import matplotlib.pyplot as plt
import sys
sy.logger.add(sys.stdout)

In [ ]:
#Starting Duet Data Owner instance
duet = sy.launch_duet()

🎤  🎸  ♪♪♪ Starting Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!
♫♫♫ > Duet Server ID: b46462b998a5a56ead9b311963fd6363

♫♫♫ > STEP 1: Send the following code to your Duet Partner!

import syft as sy
duet = sy.duet("b46462b998a5a56ead9b311963fd6363")

♫♫♫ > STEP 2: Ask your partner for their Client ID and enter it below!
♫♫♫ > Duet Partner's Client ID: 2e0cc9031e827b75f4e4bce36e4df3a1

♫♫♫ > Connecting...


/usr/local/lib/python3.7/dist-packages/aiortc/rtcdtlstransport.py:211: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  _openssl_assert(lib.SSL_CTX_use_certificate(ctx, self._cert._x509) == 1)  # type: ignore
/usr/local/lib/python3.7/dist-packages/aiortc/rtcdtlstransport.py:186: CryptographyDeprecationWarning: This version of cryptography contains a temporary pyOpenSSL fallback path. Upgrade pyOpenSSL now.
  value=certificate_digest(self._cert._x509),  # type: ignore



♫♫♫ > CONNECTED!

[2022-01-15T16:54:32.560633+0000][ERROR][logger]][60] Got an exception in Duet push. 


## Evaluating the Logistic Regression on Encrypted Data

In [ ]:
#Prepare the data
#We now prepare the training and test data, the dataset was downloaded from Kaggle.

#This dataset provides patients' information along with a 10-year risk of future coronary heart disease (CHD) as a label, and the goal is to build a model that can predict this 10-year CHD risk based on patients' information

from syft.util import get_root_data_path

def split_train_test(x, y, test_ratio=0.3):
    idxs = [i for i in range(len(x))]
    random.shuffle(idxs)
    # delimiter between test and train data
    delim = int(len(x) * test_ratio)
    test_idxs, train_idxs = idxs[:delim], idxs[delim:]
    return x[train_idxs], y[train_idxs], x[test_idxs], y[test_idxs]
            
def heart_disease_data():
    data = pd.read_csv(f"./data.csv")
    data.head()
    # drop rows with missing values
    data = data.dropna()
    # drop some features
    data = data.drop(columns=["education", "currentSmoker", "BPMeds", "diabetes", "diaBP", "BMI"])
    # balance data
    grouped = data.groupby('TenYearCHD')
    data = grouped.apply(lambda x: x.sample(grouped.size().min(), random_state=73).reset_index(drop=True))
    # extract labels
    y = torch.tensor(data["TenYearCHD"].values).float().unsqueeze(1)
    data = data.drop("TenYearCHD", 'columns')
    # standardize data
    data = (data - data.mean()) / data.std()
    x = torch.tensor(data.values).float()
    return split_train_test(x, y)


x_train, y_train, x_test, y_test = heart_disease_data()

FileNotFoundError: ignored

In [ ]:
#Making Training data Referenceable over Duet
x_train_ptr = x_train.send(duet, pointable=True, tags=["x_train"])
y_train_ptr = y_train.send(duet, pointable=True, tags=["y_train"])

In [ ]:
#Look for the requests
duet.requests.pandas

""


In [ ]:
#Approve the requests
duet.requests[0].accept()
duet.requests[0].accept()